In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
api_key = ""

In [2]:
import autogen
from autogen.coding import LocalCommandLineCodeExecutor #로컬 환경에서 실행.  만약 도커 컨테이너에서 실행하려면 DockerContainerCodeExecutor를 사용

config_list = [{"model": "gpt-4o-mini", "api_key": api_key}]

In [4]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,
        "temperature": 0,#답변이 일관성있게 나옴
    },
)
user_proxy=autogen.UserProxyAgent(
    name="user_proxy",
    # max_consecutive_auto_reply=10, #루프에 빠지지 않고 연속된 자동응답의 최대 횟수, 휴먼 인풋 모드가 NEVER인 경우에만 사용
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), #컨텐트가 TERMINATE로 끝나면 유저 프록시를 종료하는 함수
    code_execution_config={
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"), #코딩 폴더에 저장
    },
    human_input_mode="ALWAYS"
)
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.""",
    summary_method="reflection_with_llm",
)

[autogen.oai.client: 09-29 23:14:57] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
user_proxy (to assistant):

100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.

--------------------------------------------------------------------------------
assistant (to user_proxy):

아래의 파이썬 코드를 사용하여 100까지의 소수를 찾을 수 있습니다. 이 코드는 2부터 100까지의 모든 숫자를 반복하며, 각 숫자가 소수인지를 확인합니다. 소수는 1과 자기 자신 외에는 어떤 수로도 나누어 떨어지지 않는 수를 말합니다.

```python
def find_primes(n):
    primes = []
    for possiblePrime in range(2, n + 1):
        isPrime = True
        for num in range(2, int(possiblePrime ** 0.5) + 1):
            if possiblePrime % num == 0:
                isPrime = False
                break
        if isPrime:
            primes.append(possiblePrime)
    return primes

print(find_primes(100))
```

이 코드를 실행하면 100까지의 모든 소수를 출력합니다.

--------------------------------------------------------------------------------


In [5]:
print("Chat History:", chat_res.chat_history)
print("-"*50)
print("Summary:", chat_res.summary)
print("-"*50)
print("Cost Info:", chat_res.cost)


Chat History: [{'content': '100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': '아래의 파이썬 코드를 사용하여 100까지의 소수를 찾을 수 있습니다. 이 코드는 2부터 100까지의 모든 숫자를 반복하며, 각 숫자가 소수인지를 확인합니다. 소수는 1과 자기 자신 외에는 어떤 수로도 나누어 떨어지지 않는 수를 말합니다.\n\n```python\ndef find_primes(n):\n    primes = []\n    for possiblePrime in range(2, n + 1):\n        isPrime = True\n        for num in range(2, int(possiblePrime ** 0.5) + 1):\n            if possiblePrime % num == 0:\n                isPrime = False\n                break\n        if isPrime:\n            primes.append(possiblePrime)\n    return primes\n\nprint(find_primes(100))\n```\n\n이 코드를 실행하면 100까지의 모든 소수를 출력합니다.', 'role': 'user', 'name': 'assistant'}]
--------------------------------------------------
Summary: The Python code provided can be used to find all prime numbers up to 100. A prime number is a number that has no divisors other than 1 and itself. The code works by iterating through all numbers from 2 to 100 and checking 

In [6]:
user_proxy.send( #대화에 이어서 다음 메시지를 보내는 것임.
    recipient=assistant,
    message="예시 영어 문장을 만들고, 키워드를 추출하세요.",
)

user_proxy (to assistant):

예시 영어 문장을 만들고, 키워드를 추출하세요.

--------------------------------------------------------------------------------


KeyboardInterrupt: 